# AI Intern Task

## Angadi Abhinay

### Step-1: Taking a photograph using laptop/desktop camera.

In [17]:
import cv2
import dlib
import numpy as np

def capture_photo():
    # Open the camera
    cap = cv2.VideoCapture(0)

    # Checking if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Camera not found.")
        return

    # Read a frame from the camera
    ret, frame = cap.read()

    # Checking if the frame is captured successfully
    if ret:
        # Saving the captured frame as an image
        cv2.imwrite("captured_photo.jpg", frame)
        print("Photo captured and saved as 'captured_photo.jpg'.")

        # Displaying the captured image
        cv2.imshow("Captured Photo", frame)
        cv2.waitKey(0)  
        cv2.destroyAllWindows()

        # Analyzing the face in the photograph
        face_descriptor, face_landmarks = analyze_face(frame)

        # Starting video recording with face and eye movement analysis
        record_video(face_descriptor, face_landmarks)

    else:
        print("Error: Unable to capture a photo.")

    # at end stop the camera
    cap.release()

### Step-2: Analysing the face in the photograph (Throw an error if face is not found in photograph)

In [18]:
def analyze_face(image):
    # using the pre-trained cascade face detection classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Converting the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detecting faces in the image
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.3, minNeighbors=5)


    if len(faces) > 0:
        # Use the first face for simplicity
        (x, y, w, h) = faces[0]

        # Extracting the face for region of interest
        face_roi = gray_image[y:y+h, x:x+w]

        # Loading the shape predictor model
        shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        # Using dlib for facial landmarks
        shape = shape_predictor(image, dlib.rectangle(x, y, x+w, y+h))
        face_landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])

        # Using dlib for face recognition
        face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
        face_descriptor = face_recognizer.compute_face_descriptor(image, shape)

        return face_descriptor, face_landmarks

    else:
        print("Error: No face found in the photograph.")
        return None, None

### Step-3: Next, video recording will start, match the photo and the face in the video continuously until the video recording stops. If the face does not match with the photo, throw an error.

In [19]:
def record_video(reference_face_descriptor, reference_face_landmarks):
    # Open the default camera for video recording
    cap = cv2.VideoCapture(0)

    # Checking if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Camera not found.")
        return

    # Defining the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (640, 480))

    while True:
        # Reading a frame from the camera
        ret, frame = cap.read()

        # Analyzing the face and eye movement in the video frame
        current_face_descriptor, current_face_landmarks = analyze_face(frame)

        # Checking if face is found in the current frame
        if current_face_descriptor is not None:
            # Comparing the current face with the reference face
            match = compare_faces(reference_face_descriptor, current_face_descriptor)

            # If the faces do not match, throw an error
            if not match:
                print("Error: Face in the video does not match the reference face.")
                break

            # Analyzing eye movement
            analyze_eye_movement(reference_face_landmarks, current_face_landmarks, frame)

        # Writing the frame to the output video
        out.write(frame)

        # Displaying the frame
        cv2.imshow('Video Recording', frame)

        # stop the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Releasing everything when done
    cap.release()
    out.release()
    cv2.destroyAllWindows()

### Step 4: Analyse eye movement in the video, whether he is looking into the screen or outside the screen. When he is looking outside the screen, throw a warning.

In [20]:
def compare_faces(reference_face_descriptor, current_face_descriptor, threshold=0.6):
    # Convert face descriptors to NumPy arrays
    reference_np = np.array(reference_face_descriptor)
    current_np = np.array(current_face_descriptor)

    # Compare face descriptors using Euclidean distance
    distance = np.linalg.norm(reference_np - current_np)

    # Determine if the faces match based on the threshold
    return distance < threshold

def analyze_eye_movement(reference_landmarks, current_landmarks, frame):
    # Indices of eye landmarks
    left_eye_indices = [36, 37, 38, 39, 40, 41]
    right_eye_indices = [42, 43, 44, 45, 46, 47]

    # Extract coordinates of eye landmarks
    left_eye_ref = reference_landmarks[left_eye_indices]
    right_eye_ref = reference_landmarks[right_eye_indices]
    left_eye_current = current_landmarks[left_eye_indices]
    right_eye_current = current_landmarks[right_eye_indices]

    # Calculate the average position of the eyes in the reference frame
    avg_left_eye_ref = np.mean(left_eye_ref, axis=0)
    avg_right_eye_ref = np.mean(right_eye_ref, axis=0)

    # Calculate the average position of the eyes in the current frame
    avg_left_eye_current = np.mean(left_eye_current, axis=0)
    avg_right_eye_current = np.mean(right_eye_current, axis=0)

    # Calculate the distance between the average eye positions
    distance_ref = np.linalg.norm(avg_left_eye_ref - avg_right_eye_ref)
    distance_current = np.linalg.norm(avg_left_eye_current - avg_right_eye_current)

    # Set a threshold for eye movement (you may need to adjust this based on your observations)
    eye_movement_threshold = 10.0

    # Compare the distances and determine if there is significant eye movement
    if abs(distance_current - distance_ref) > eye_movement_threshold:
        cv2.putText(frame, "Warning: Eyes off- the screen!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

if __name__ == "__main__":
    capture_photo()


Photo captured and saved as 'captured_photo.jpg'.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the photograph.
Error: No face found in the ph